### Set up the right directory for importing modules

In [ ]:
import sys
from pathlib import Path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [ ]:
import src.CA_model as CA
import src.analysis as an
import time
import matplotlib.pyplot as plt
import numpy as np
import importlib

In [ ]:
# run to reload CA_model.py and analysis.py for updated code
importlib.reload(CA)
importlib.reload(an)

#### Parameter settings

In [ ]:
size = 100                          # width and height of the grid
p = 0.5                             # starting fraction of vegetation
update_rule = CA.update_Scanlon2007 # function containing update rule
true_frac=0.2                       # 'natural' (equilibrium) fraction of vegetation
k=3                                 # strength of local interactions
M=20                                # radius of neighborhood
N_steps=200                         # number of iterations
skip=0                              # iterations to skip (equilibration period)

#### Generate grids for analysis

In [ ]:
grids = CA.evolve_CA(
    size=size,
    p=p,
    update_rule=update_rule,
    true_frac=true_frac,
    k=k,
    M=M,
    N_steps=N_steps,
    skip=skip,
)

#### Plot starting and final grid

In [ ]:
an.show_grids([grids[0], grids[-1]], [skip, N_steps])

#### Plot cluster size distribution

In [ ]:
size_list, fit = an.cluster_sizes(grids[100:])
beta = fit.truncated_power_law.alpha
s_char = 1 / (fit.truncated_power_law.Lambda)
R, p = fit.distribution_compare("truncated_power_law", "exponential")

print("Scaling exponent: ", beta)
print("Characteristic length: ", s_char)
print("R: ", R)
print("p?: ", p)

In [ ]:
an.plot_cluster_size_distr(size_list=size_list, fit=fit)

## Averaging over several datasets

In [ ]:
size = 200                          # width and height of the grid
p = 0.5                             # starting fraction of vegetation
update_rule = CA.update_Scanlon2007 # function containing update rule
true_frac=0.2                       # 'natural' (equilibrium) fraction of vegetation
k=3                                 # strength of local interactions
M=20                                # radius of neighborhood
N_steps=200                         # number of iterations
skip=100                            # iterations to skip (equilibration period)

N_evolutions = 5                    # number of full evolutions to generate for this set of parameters
all_grids = []

for seed in range(N_evolutions):
    start = time.time()
    grids = CA.evolve_CA(
        size=size,
        p=p,
        update_rule=update_rule,
        true_frac=true_frac,
        k=k,
        M=M,
        N_steps=N_steps,
        skip=skip,
        seed=seed,
    )
    all_grids.append(grids)
    end = time.time()
    print(f"Grid evolution {seed+1} out of {N_evolutions} completed in {end-start} seconds.")


In [ ]:
# flatten the list of lists into a 1D list of grids
combined_grids = [grid for grid_list in all_grids for grid in grid_list]

In [ ]:
size_list, fit = an.cluster_sizes(combined_grids)
beta = fit.truncated_power_law.alpha
s_char = 1 / (fit.truncated_power_law.Lambda)
R, p = fit.distribution_compare("truncated_power_law", "exponential")

print("Scaling exponent: ", beta)
print("Characteristic length: ", s_char)
print("R: ", R)
print("p?: ", p)

In [ ]:
an.plot_cluster_size_distr(size_list=size_list, fit=fit)